In [1]:
import os
import numpy as np
import nibabel as nib
import pandas as pd
from nilearn import datasets, input_data
from nilearn.image import resample_to_img

In [2]:
TR = 1.5 # repition time (time that each image takes up)

In [3]:
# Load Schaefer 600 parcellation atlas
schaefer = datasets.fetch_atlas_schaefer_2018(n_rois=600, resolution_mm=2, yeo_networks=7)
atlas_img = nib.load(schaefer.maps)
atlas_labels = schaefer.labels

[get_dataset_dir] Dataset found in C:\Users\ekkid\nilearn_data\schaefer_2018

In [4]:
# Load all fMRI images in the data directory (taking from subject 1 and averaging all 6 test trials)
directory = "data"

fmri_images = []
timestamps_list = []
for filename in os.listdir(directory):
    if filename.endswith(".nii"):
        filepath = os.path.join(directory, filename)
        img = nib.load(filepath)
        # Append the image to a list of the images that I would like to use.
        fmri_images.append(img)
    if filename.endswith(".tsv"):
        filepath = os.path.join(directory, filename)
        stamp = pd.read_csv(filepath, sep="\t")
        timestamps_list.append(stamp)

In [5]:
reference_img = fmri_images[0]

# Resample all fMRI images to match the first one. Allows for all the images to be the same shape.
resampled_fmri_images = []
for img in fmri_images:
    resampled_fmri_images.append(resample_to_img(img, reference_img, interpolation="linear", copy_header=True, force_resample=True))
    print(f"Done resampling")

Done resampling
Done resampling
Done resampling
Done resampling
Done resampling
Done resampling


In [6]:
from nilearn import image, input_data

# Resample atlas to match the first fMRI image
reference_img = fmri_images[0]
resampled_atlas = image.resample_to_img(
    atlas_img, reference_img, interpolation="nearest", copy_header=True, force_resample=True
)

# Save the resampled atlas for consistency
resampled_atlas.to_filename("resampled_atlas.nii.gz")

# Use this resampled atlas consistently
masker = input_data.NiftiLabelsMasker(resampled_atlas, standardize=True)


In [7]:
# Now process all fMRI images with the same masker
fmri_series = []
for img in resampled_fmri_images:
    series = masker.fit_transform(img)
    fmri_series.append(series)
    print(f"Done Series: {series.shape}")

Done Series: (410, 574)
Done Series: (410, 574)
Done Series: (410, 574)
Done Series: (410, 574)
Done Series: (410, 574)
Done Series: (410, 574)


In [8]:
genre_indices_list = []
genres = []
# Loop through all stamps
for stamps in timestamps_list:
    genre_fmri_indices = {}
    
    # Loop through all rows
    for _, row in stamps.iterrows():
        genre = row['genre']
        onset = row['onset']
        duration = row ['duration']

        # Compute fMRI indices
        start_index = int(onset / TR)
        end_index = int((onset + duration) / TR)
        
        # Create list of fMRI indices in this range
        indices = list(range(start_index, end_index))
        
        # Store in dictionary
        if genre in genre_fmri_indices:
            genre_fmri_indices[genre].extend(indices)
        else:
            genre_fmri_indices[genre] = indices

        if genre not in genres:
            genres.append(genre)

    # Remove duplicates and sort indices for each genre
    for genre in genre_fmri_indices:
        genre_fmri_indices[genre] = sorted(set(genre_fmri_indices[genre]))

    genre_indices_list.append(genre_fmri_indices)


In [9]:
genre_fmri_indices = {}
# Loop through all rows
for _, row in timestamps_list[0].iterrows():
    genre = row['genre']
    onset = row['onset']
    duration = row['duration']
    
    # Compute fMRI indices
    start_index = int(onset / TR)
    end_index = int((onset + duration) / TR)
    
    # Create list of fMRI indices in this range
    indices = list(range(start_index, end_index))
    
    # Store in dictionary
    if genre in genre_fmri_indices:
        genre_fmri_indices[genre].extend(indices)
    else:
        genre_fmri_indices[genre] = indices

# Remove duplicates and sort indices for each genre
for genre in genre_fmri_indices:
    genre_fmri_indices[genre] = sorted(set(genre_fmri_indices[genre]))

pop_time_series = fmri_series[0][genre_fmri_indices["'pop'"]]
print(pop_time_series)

# Compute correlation matrix
correlation_matrix = np.corrcoef(pop_time_series.T)
print(correlation_matrix.shape)

[[-2.05209751 -1.48261402 -2.14670053 ... -0.99089043 -1.35943729
  -1.32067788]
 [-2.14916395 -1.81790213 -2.54322959 ... -0.95848848 -1.6221799
  -1.54232861]
 [-1.92992322 -2.60770969 -2.21530635 ... -1.39539221 -1.62791842
  -1.40026854]
 ...
 [ 1.55282058  2.14794008  1.25904346 ...  2.49702287  1.88324093
   1.47286172]
 [ 1.68819591  0.05218932  1.56808436 ...  2.87957494  2.24558799
   1.88377163]
 [ 1.81502939  1.2789117   1.63731959 ...  2.30052071  2.27387073
   1.93652358]]
(574, 574)


In [10]:
# Get unique labels (nonzero ones)
masked_atlas_labels = np.unique(resampled_atlas.get_fdata().astype(int))
masked_atlas_labels = masked_atlas_labels[masked_atlas_labels > 0]  # Exclude background (zero)

# Get the corresponding label names
remaining_label_names = [atlas_labels[label - 1] for label in masked_atlas_labels]  # -1 to index correctly

# Print results
print(f"Number of labels after resampling: {len(remaining_label_names)}")
print("Remaining labels:", remaining_label_names[:10])  # Print first 10 for verification


Number of labels after resampling: 574
Remaining labels: [b'7Networks_LH_Vis_1', b'7Networks_LH_Vis_2', b'7Networks_LH_Vis_3', b'7Networks_LH_Vis_4', b'7Networks_LH_Vis_5', b'7Networks_LH_Vis_6', b'7Networks_LH_Vis_7', b'7Networks_LH_Vis_9', b'7Networks_LH_Vis_10', b'7Networks_LH_Vis_11']


In [ ]:
brain_regions = []
# Do this for each genre.
for genre in genres:
    genre_series_list = []
    for i in range(len(fmri_series)):
        series = fmri_series[i]
        indices = genre_indices_list[i]
        genre_series_list.append(series[indices[genre]])  # Collect relevant time series

    genre_series = np.vstack(genre_series_list)  # Final shape: (250, 574)
    # Create a correlation matrix from the total time series
    correlation_matrix = np.corrcoef(genre_series.T)
    #average_matrix = matrices[0]

    # Create the node list for this genre
    parsed_labels = []
    for label in remaining_label_names:
        label = label.decode("utf-8")  # Convert bytes to string
        parts = label.split("_")
        
        network = parts[0]  # "7Networks"
        hemisphere = parts[1]  # "LH" or "RH"
        region = "_".join(parts[2:-1])  # Join everything in between as the region
        parcel_id = parts[-1]  # Last part is the parcel number

        brain_region = {"hemisphere": hemisphere, "Region":region}
        if not brain_region in brain_regions:
            brain_regions.append(brain_region)

        parsed_labels.append([label, network, hemisphere, region, parcel_id])

    # Convert to DataFrame
    node_list = pd.DataFrame(parsed_labels, columns=["Label", "Network", "Hemisphere", "Region", "Parcel ID"])
    node_list.insert(0, "Node", range(1, len(node_list) + 1))  # Add Node index

    # Save to CSV inside the 'networks' folder
    node_list.to_csv(f"networks/{genre}_node_list.csv", index=False)

    # Create edge list for correlations greater than 0.7
    edges = []
    n_nodes = correlation_matrix.shape[0]
    for i in range(n_nodes):
        for j in range(i + 1, n_nodes):  # Avoid duplicates
            # Add all edges to the graph
            edges.append((i + 1, j + 1, correlation_matrix[i, j]))

    # Sort edges by correlation value (descending order)
    edges.sort(key=lambda x: x[2], reverse=True)

    # Select top k edges
    k = 6000  # Define fixed number of edges. FIXED DENSITY
    edges = edges[:k]  # Take the top-k strongest edges

    edge_list = pd.DataFrame(edges, columns=["Node1", "Node2", "Correlation"])
    edge_list.to_csv(f"networks/{genre}_edge_list.csv", index=False)

    print(f"Node list and edge list for {genre} have been saved.")

AttributeError: 'DataFrame' object has no attribute 'sort'

In [ ]:
brain_regions

[{'hemisphere': 'LH', 'Region': 'Vis'},
 {'hemisphere': 'LH', 'Region': 'SomMot'},
 {'hemisphere': 'LH', 'Region': 'DorsAttn_Post'},
 {'hemisphere': 'LH', 'Region': 'DorsAttn_FEF'},
 {'hemisphere': 'LH', 'Region': 'DorsAttn_PrCv'},
 {'hemisphere': 'LH', 'Region': 'SalVentAttn_ParOper'},
 {'hemisphere': 'LH', 'Region': 'SalVentAttn_TempOcc'},
 {'hemisphere': 'LH', 'Region': 'SalVentAttn_FrOperIns'},
 {'hemisphere': 'LH', 'Region': 'SalVentAttn_PFCl'},
 {'hemisphere': 'LH', 'Region': 'SalVentAttn_Med'},
 {'hemisphere': 'LH', 'Region': 'Limbic_OFC'},
 {'hemisphere': 'LH', 'Region': 'Limbic_TempPole'},
 {'hemisphere': 'LH', 'Region': 'Cont_Par'},
 {'hemisphere': 'LH', 'Region': 'Cont_Temp'},
 {'hemisphere': 'LH', 'Region': 'Cont_PFCd'},
 {'hemisphere': 'LH', 'Region': 'Cont_OFC'},
 {'hemisphere': 'LH', 'Region': 'Cont_PFCl'},
 {'hemisphere': 'LH', 'Region': 'Cont_PFCv'},
 {'hemisphere': 'LH', 'Region': 'Cont_pCun'},
 {'hemisphere': 'LH', 'Region': 'Cont_Cing'},
 {'hemisphere': 'LH', 'Regio